In [1]:
import psycopg2
import pandas as pd
from reportlab.lib.pagesizes import landscape
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch

# Custom page size (11.00 x 8.50 inches)
custom_page_size = (11.00 * inch, 8.50 * inch)

def draw_header(c, width, height, page_num, headers, header_x_positions):
    c.setFont("Courier", 10.5)
    c.drawString(10, height - 20, "REPORT: CSWR602")
    c.drawString(250, height - 20, "OPEN PO EXCEPTIONS BETWEEN ORACLE AND BICEPS")
    c.drawString(width - 90, height - 20, f"PAGE {page_num}")
    c.drawString(10, height - 35, "DATE: 02/19/24")
    c.drawString(width - 90, height - 35, "TIME 01.43.05")

    # Draw table header
    c.setFont("Courier", 10.5)
    for i, header in enumerate(headers):
        c.drawString(header_x_positions[i], height - 55, header)

def create_pdf(file_name, df):
    c = canvas.Canvas(file_name, pagesize=custom_page_size)
    width, height = custom_page_size

    # Pagination variables
    row_height = 12
    max_rows_per_page = int((height - 70) / row_height)
    num_pages = -(-len(df) // max_rows_per_page)  # Ceiling division

    # Column headers and positions
    headers = df.columns.tolist()
    header_x_positions = [5, 50, 110, 170, 200, 220, 290, 360, 430, 460, 510, 560]  # Adjusted x-coordinates

    for page_num in range(1, num_pages + 1):
        draw_header(c, width, height, page_num, headers, header_x_positions)

        # Determine the start and end indices of the rows to draw
        start_idx = (page_num - 1) * max_rows_per_page
        end_idx = start_idx + max_rows_per_page

        # Draw the rows
        rows_to_draw = df.iloc[start_idx:end_idx]
        start_y = height - 70

        for _, row in rows_to_draw.iterrows():
            for i, header in enumerate(headers):
                c.drawString(header_x_positions[i], start_y, str(row[header]))
            start_y -= row_height

        # Create a new page if not on the last page
        if page_num < num_pages:
            c.showPage()

    c.save()

def fetch_data_from_postgres():
    # Database credentials
    user = "cspoms"
    password = "cspoms"
    host = "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com"
    port = "5432"
    database = "CSPODB"
    sslmode = "verify-ca"
    sslcert= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-cert.pem"
    sslkey= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-key.pem"
    sslrootcert= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\server-ca.pem"

    # Initialize cursor and connection to None
    connection = None
    cursor = None

    try:
        # Establish the connection
        connection = psycopg2.connect(
            database=database,
            user=user,
            password=password,
            host=host,
            port=port,
            sslmode=sslmode,
            sslcert=sslcert,
            sslkey=sslkey,
            sslrootcert=sslrootcert
        )

        # Create a cursor object
        cursor = connection.cursor()

        # Execute a query to fetch all rows from the table
        query = '''SELECT
        po_num AS "ORDER",
        secondary_po AS "SECONDARY",
        recon_origin AS "ORIGIN", -- keeping the original recon_origin column
        po_oss_dest AS "DST",
        po_type AS "TP",
        order_date AS "ORDERED",
        due_date AS "ARRIVAL", 
        order_date AS "APPT", 
        bic_buyer_id AS "BYR",
        vendor_num AS "VENDOR",
        CASE
            WHEN recon_origin = 'O' THEN 'oracle'
            WHEN recon_origin = 'M' THEN 'mainframe'
        END AS "SOURCE", 
        error_message AS "MESSAGE"
    FROM
        xxpo_po_recon_inbound_stg xpris
    WHERE
        xpris.process_status = 'E'
        AND xpris.request_id = 999;'''  

        # Execute the query
        cursor.execute(query)

        # Fetch all rows from the result of the query
        rows = cursor.fetchall()

        # Get column names
        column_names = [desc[0] for desc in cursor.description]

        # Create a DataFrame
        df = pd.DataFrame(rows, columns=column_names)
        return df

    finally:
        # Clean up and close the connection
        if cursor:
            cursor.close()
        if connection:
            connection.close()

# Fetch data from PostgreSQL
df = fetch_data_from_postgres()

# Repeat the DataFrame rows 5 times
df = pd.concat([df], ignore_index=True)

# Generate PDF with the fetched data
create_pdf("CSWR602_ReportY.pdf", df)

In [1]:
from datetime import datetime

def extract_timestamp(full_time_str):
    return datetime.strptime(full_time_str, '%Y%m%d%H%M%S.%f')

# Example usage in sorting
fileList = [...]  # List of file names or strings containing timestamps
sorted_fileList = sorted(fileList, key=extract_timestamp)


TypeError: strptime() argument 1 must be str, not ellipsis

In [7]:
pip install reportlab

   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB 165.2 kB/s eta 0:00:12
   ---------------------------------------- 0.0/1.9 MB 165.2 kB/s eta 0:00:12
    --------------------------------------- 0.0/1.9 MB 146.3 kB/s eta 0:00:14
    --------------------------------------- 0.0/1.9 MB 164.3 kB/s eta 0:00:12
    --------------------------------------- 0.0/1.9 MB 164.3 kB/s eta 0:00:12
   - -------------------------------------- 0.1/1.9 MB 124.9 kB/s eta 0:00:16
   - -------------------------------------- 0.1/1.9 MB 136.5 kB/s eta 0:00:14
   - -------------------------------------- 0.1/1.9 MB 136.5 kB/s eta 0:00:14
   - -------------------------------------- 0.1/1.9 MB 164.0 kB/s eta 0:00:12
   -- ------------------------------------- 0.1/1.9 MB 204.8 kB/s eta 0:00:09
   -- ------------------------------------- 0.1/1.9 MB 204.8 kB/s eta 0:00:09
   -